# Reflexion

#### Goal: Enhance emergency housing allocation.


#### Caracteristics of the model:
- Accuracy:
1. if used as a clearing tool removing overburden upfront, and thus only to get rid of obvious cases, the accuracy of such a tool could be its most important caracteristic.

- Interpretability:
1. could help families understand the decision (although not as important as in diseases predictions).
2. can also highlight and thus control biases (racial, sex, age).
3. since the tool would probably be used in combination with human selection, it could help save time by highlighting the main factors for each decision

#### Conclusion:
- a model easily interpretable could be prefered (tree).
- or a highly accurate model (less interpretable) could also be used upfront (NN).

## Functions & imports

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import log_loss
from sklearn import tree

from cobratools import Analysis

In [2]:
# Define the test scorer
def competition_scorer(y_true, y_pred):
    return log_loss(y_true, y_pred, sample_weight=10**y_true)

# Transform methods
def num_to_onehot(arr):
    """ Transform a numeric categorical vector into a one-hot matrix """
    arr_onehot = np.zeros((arr.size, arr.max()+1))
    arr_onehot[np.arange(arr.size), arr] = 1
    return arr_onehot

def onehot_to_num(arr):
    """ Transform a one-hot encoded matrix into a numeric categorical vector """
    arr_num = arr.dot(np.arange(arr.shape[1]))
    return arr_num

# Analytics
def count_pred_errors(arr_preds, arr_truth):
    """ Counts the number of wrong predictions """
    return sum(arr_preds != arr_truth)

## Load data

In [3]:
requests_train = pd.read_csv(filepath_or_buffer='data/requests_train.csv',
                             sep=',',
                             low_memory=False,
                             error_bad_lines=False)

requests_test = pd.read_csv(filepath_or_buffer='data/requests_test.csv',
                            sep=',',
                            low_memory=False,
                            error_bad_lines=False)

individuals_train = pd.read_csv(filepath_or_buffer='data/individuals_train.csv',
                                sep=',',
                                low_memory=False,
                                error_bad_lines=False)

individuals_test = pd.read_csv(filepath_or_buffer='data/individuals_test.csv',
                               sep=',',
                               low_memory=False,
                               error_bad_lines=False)

# Analyze data

In [ ]:
Analyze = Analysis(requests_train)
Analyze.describe(investigation_level=3)
Analyze.visualize()

# Predict

## Build models

### Benchmarks

In [118]:
# Random uniform train/test
random_preds_train = np.random.uniform(size=(requests_train.shape[0], 4))
random_preds_test = np.random.uniform(size=(requests_test.shape[0], 4))

# Dumb (always pred 3)
dumb_preds_train = np.zeros((requests_train.shape[0], 4))
dumb_preds_test = np.zeros((requests_test.shape[0], 4))
# Set 10% pred everywhere (if not, log penalyzes hardly)
dumb_preds_train[:,:] = .01
dumb_preds_test[:,:] = .01
# Set 20% pred on class 3
dumb_preds_train[:,2] = .02
dumb_preds_test[:,2] = .02

### Univariate predictions

We observed a significant (negative) correlation of housing_situation_id with granted_number_of_nights, let's train a univariate model


In [128]:
# Set model' parameters
clf = tree.DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    max_depth=2,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=None,
    max_leaf_nodes=None,
    class_weight=None)

# Build train/test datasets with housing_situation_id only
X_train = np.array(requests_train['housing_situation_id']).reshape(-1, 1)
X_test = np.array(requests_test['housing_situation_id'].values).reshape(-1, 1)
Y_train = requests_train.granted_number_of_nights.values
Y_test = requests_test.granted_number_of_nights.values

# Transform categorical target into a one-hot vector
Y_train_onehot = to_onehot(Y_train)
Y_test_onehot = to_onehot(Y_test)

# Train the model
clf = clf.fit(X_train, Y_train_onehot)

# Yield train/test predictions
preds_train_tree_univar = clf.predict(X_train)
preds_test_tree_univar = clf.predict(X_test)

# Fill predictions to .2 elsewhere
preds_train_tree_univar[preds_train_tree_univar == 0] = .2
preds_test_tree_univar[preds_test_tree_univar == 0] = .2

# Evaluate train/test
score_train = competition_scorer(Y_train, preds_train_tree_univar)
score_test = competition_scorer(Y_test, preds_test_tree_univar)

# Display results
print(f'train score: {score_train:.2f}')
print(f'test score: {score_test:.2f}')

train score: 1.72
test score: 1.71


In [102]:
probas = clf.predict_proba(X_train)
v0 = probas[0].max(1)
v1 = probas[1].max(1)
v2 = probas[2].max(1)
v3 = probas[3].max(1)

## Evaluate models

In [122]:
y_true_test = requests_test.granted_number_of_nights.values

# Evaluate benchmarks
random_score_test = competition_scorer(y_true_test, random_preds_test)
dumb_score_test = competition_scorer(y_true_test, dumb_preds_test)

# Display results
print(f'test score random: {random_score_test:.2f}')
print(f'test score dumb: {dumb_score_test:.2f}')

test score random: 1.67
test score dumb: 1.25


### Train set

In [120]:
y_true_train = requests_train.granted_number_of_nights.values

# Evaluate benchmarks
random_score_train = competition_scorer(y_true_train, random_preds_train)
dumb_score_train = competition_scorer(y_true_train, dumb_preds_train)

# Display results
print(f'train score random: {random_score_train:.2f}')
print(f'train score dumb: {dumb_score_train:.2f}')

train score random: 1.65
train score dumb: 1.25


### Test set

## Interpret models

In [ ]:
# Tree
fn = ['housing_situation_id']
cn = ['0', '1', '2', '3']

tree.plot_tree(clf,
               feature_names = fn, 
               class_names=cn,
               filled = True)

requests_train['housing_situation_id'].hist()